In [2]:
def f(a=[]):
  a.append("fuck")
  return a

print(f())
print(f())

['fuck']
['fuck', 'fuck']


In [5]:

def construct_prompt(sample):
    question_prompt = "Question:\n" \
                      "{question}\n".format(question=sample["input"])

    # options = [
    #     (chr(ord("A") + i), opt)
    #     for i, opt in enumerate(sample["options"])
    # ]
    options = sample["options"]
    option_prompt = "Options:\n" + "\n".join(["{k}) {v}".format(k=chr(k + ord("A")), v=v) for k, v in enumerate(options)]) + "\n"

    system_prompt = "Imagine you are a doctor, and please suggest the treatmeat based on the given patient information above. You need to choose one option from the following list.\n"
    instruction_prompt = "Answer only with the option index such as A/B/C/D/E/F/G in plain text."
    cot_instruction_prompt = "Reason step-by-step before answering. Answer only with the option index such as A/B/C/D/E/F/G in plain text. Your final output should strictly follow this format:\n" \
                             "<Reason>{your step-by-step reasoning}</Reason><Answer>{your answer}</Answer>"

    prompt = "\n".join([question_prompt, system_prompt, option_prompt, instruction_prompt])
    prompt_with_cot = "\n".join(
        [question_prompt, system_prompt, option_prompt, cot_instruction_prompt]
    )

    return {
        "prompt": prompt,
        "prompt_with_cot": prompt_with_cot
    }

In [6]:
sample = {
    "input": "{question}",
    "options": {"A": "A", "B": "B"}
}

In [8]:
print(construct_prompt(sample)["prompt"])

Question:
{question}

Imagine you are a doctor, and please suggest the treatmeat based on the given patient information above. You need to choose one option from the following list.

Options:
A) A
B) B

Answer only with the option index such as A/B/C/D/E/F/G in plain text.


In [2]:
import pandas as pd

In [3]:
# Let's recreate the data carefully to ensure all columns have the correct length

data = {
    "Task": [
        "Concept Recognition", "Concept Recognition", "Concept Recognition", "Concept Recognition", 
        "Concept Recognition", "Concept Recognition", "Concept Recognition", "Clinical Decision Support", 
        "Clinical Decision Support", "Clinical Decision Support", "Clinical Decision Support", 
        "Clinical Decision Support", "Reasoning", "Reasoning", "Reasoning", "Reasoning", 
        "Knowledge", "Knowledge", "Knowledge", "Knowledge", "Knowledge", "Mathematical"
    ],
    "Dataset": [
        "PMC-Patient", "PICO-Participant", "PICO-Intervention", "PICO-Outcome", 
        "ADE Corpus", "BC5-disease", "BC5Chem", "DDXPlus", 
        "SEER", "MIMIC4ED -Hospitalization", "MIMIC4ED -72h ED Revisit", 
        "MIMIC4ED -Critical Triage", "MedNLI-Dis.", "PUBHEALTH Ver.", 
        "PubMedQA", "MedQA", "MedMCQA", "LancetQA", "NEJMQA", "Medbullets", "MedCalc-Bench"
    ],
    "o1": [
        76.4, 75.0, 77.5, 67.5, 75.0, 75.0, 71.2, 52.0, 72.4, 64.0, 
        59.7, 61.7, 88.0, 76.4, 75.0, 75.0, 95.0, 81.5, 91.2, 90.6, 34.9
    ],
    "GPT-4": [
        75.7, 75.0, 75.0, 65.0, 85.0, 85.0, 71.2, 45.0, 38.0, 61.0, 
        58.0, 66.7, 84.0, 75.7, 52.8, 69.7, 95.0, 76.1, 83.5, 66.9, 25.5
    ],
    "GPT-3.5": [
        74.4, 52.5, 75.0, 60.0, 90.0, 69.5, 43.1, 34.6, 5.0, 62.0, 
        53.6, 58.7, 57.0, 74.4, 25.4, 53.8, 88.5, 61.0, 65.0, 50.7, 10.8
    ],
    "MEDITRON* (70B)": [
        72.2, 72.1, 46.6, 51.2, 95.7, 69.6, 4.3, 29.6, 68.3, 56.3, 
        48.5, 45.7, 60.9, 32.7, 74.4, 47.9, 58.8, 0.0, None, 0.0, None
    ],
    "Llama3* (8B)": [
        96.0, 58.2, 79.1, 58.2, 69.6, 25.3, 19.5, 33.8, 56.1, 39.1, 
        9.3, 8.8, 63.9, 63.9, 73.0, 50.7, 50.7, None, None, None, None
    ]
}

In [6]:
[(t, d) for t, d in zip(data["Task"], data["Dataset"])]

[('Concept Recognition', 'PMC-Patient'),
 ('Concept Recognition', 'PICO-Participant'),
 ('Concept Recognition', 'PICO-Intervention'),
 ('Concept Recognition', 'PICO-Outcome'),
 ('Concept Recognition', 'ADE Corpus'),
 ('Concept Recognition', 'BC5-disease'),
 ('Concept Recognition', 'BC5Chem'),
 ('Clinical Decision Support', 'DDXPlus'),
 ('Clinical Decision Support', 'SEER'),
 ('Clinical Decision Support', 'MIMIC4ED -Hospitalization'),
 ('Clinical Decision Support', 'MIMIC4ED -72h ED Revisit'),
 ('Clinical Decision Support', 'MIMIC4ED -Critical Triage'),
 ('Reasoning', 'MedNLI-Dis.'),
 ('Reasoning', 'PUBHEALTH Ver.'),
 ('Reasoning', 'PubMedQA'),
 ('Reasoning', 'MedQA'),
 ('Knowledge', 'MedMCQA'),
 ('Knowledge', 'LancetQA'),
 ('Knowledge', 'NEJMQA'),
 ('Knowledge', 'Medbullets'),
 ('Knowledge', 'MedCalc-Bench')]

In [5]:
data_debug = {key: len(value) for key, value in data.items()}
data_debug

{'Task': 22,
 'Dataset': 21,
 'o1': 21,
 'GPT-4': 21,
 'GPT-3.5': 21,
 'MEDITRON* (70B)': 21,
 'Llama3* (8B)': 21}

In [4]:
df = pd.DataFrame(data)

# Calculate the mean score for each model in each aspect and task
mean_scores = df.groupby("Task").mean()

import ace_tools as tools; tools.display_dataframe_to_user(name="Mean Scores by Task", dataframe=mean_scores)


ValueError: All arrays must be of the same length